In [1]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [2]:
sample_p = pd.read_csv('./data/sample_music')
sample_p

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop,song_popularity
0,0,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,...,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4,Like a Rolling Stone,Bob Dylan,"['classic rock', 'country rock', 'folk', 'folk...",72,68
1,1,0.485,0.863,1,-9.027,1,0.0495,0.000012,0.016200,0.1380,...,spotify:track:3oTlkzk1OtrhH8wBAduVEi,https://api.spotify.com/v1/tracks/3oTlkzk1Otrh...,https://api.spotify.com/v1/audio-analysis/3oTl...,300977,4,Smells Like Teen Spirit,Nirvana,"['grunge', 'permanent wave', 'rock']",82,0
2,2,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,...,spotify:track:3ZFBeIyP41HhnALjxWy1pR,https://api.spotify.com/v1/tracks/3ZFBeIyP41Hh...,https://api.spotify.com/v1/audio-analysis/3ZFB...,337413,4,A Day In The Life - Remastered,The Beatles,"['beatlesque', 'british invasion', 'classic ro...",84,0
3,3,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,...,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...,219147,4,Good Vibrations (Mono),The Beach Boys,"['baroque pop', 'classic rock', 'psychedelic r...",78,20
4,4,0.518,0.756,10,-10.851,1,0.0915,0.735000,0.000062,0.3170,...,spotify:track:7MH2ZclofPlTrZOkPzZKhK,https://api.spotify.com/v1/tracks/7MH2ZclofPlT...,https://api.spotify.com/v1/audio-analysis/7MH2...,160893,4,Johnny B Goode,Chuck Berry,"['classic rock', 'rock', 'rock-and-roll', 'roc...",75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,1094,0.509,0.680,5,-8.354,1,0.0372,0.360000,0.000000,0.0824,...,spotify:track:7tJQ4Ekp2vN3NlI3vJJW3v,https://api.spotify.com/v1/tracks/7tJQ4Ekp2vN3...,https://api.spotify.com/v1/audio-analysis/7tJQ...,185200,4,I Want You,Bob Dylan,"['classic rock', 'country rock', 'folk', 'folk...",72,56
1095,1095,0.612,0.668,6,-6.143,0,0.0302,0.057900,0.044500,0.3410,...,spotify:track:6EaHDsTWNNJRkCMtptSE6n,https://api.spotify.com/v1/tracks/6EaHDsTWNNJR...,https://api.spotify.com/v1/audio-analysis/6EaH...,315200,4,The Suburbs,Arcade Fire,"['baroque pop', 'canadian indie', 'indie rock'...",66,0
1096,1096,0.651,0.944,2,-5.480,1,0.0381,0.085500,0.145000,0.1160,...,spotify:track:5mWhSQfWEeBZD9YTd4yyFc,https://api.spotify.com/v1/tracks/5mWhSQfWEeBZ...,https://api.spotify.com/v1/audio-analysis/5mWh...,263440,4,Dum Surfer,King Krule,['uk alternative pop'],60,3
1097,1097,0.492,0.194,6,-14.073,1,0.0577,0.984000,0.104000,0.1820,...,spotify:track:5iltwljBW1H5ScLeeiB9ZB,https://api.spotify.com/v1/tracks/5iltwljBW1H5...,https://api.spotify.com/v1/audio-analysis/5ilt...,203373,4,Black and Tan Fantasy,Duke Ellington & His Washingtonians,['hot jazz'],18,11


In [172]:
sample_p['artist_genres'][0]

"['classic rock', 'country rock', 'folk', 'folk rock', 'rock', 'roots rock', 'singer-songwriter']"

In [175]:
s = sample_p['artist_genres']
new_column=s.str.strip('""[]')

In [184]:
new_column2=new_column.str.split(",")

In [186]:
sample_p['artist_genres']=new_column2.str[0]

In [187]:
sample_p['artist_genres']

0             'classic rock'
1                   'grunge'
2               'beatlesque'
3              'baroque pop'
4             'classic rock'
                ...         
1094          'classic rock'
1095           'baroque pop'
1096    'uk alternative pop'
1097              'hot jazz'
1098                   'ska'
Name: artist_genres, Length: 1099, dtype: object

In [188]:
sample_p

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop,song_popularity
0,0,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,...,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4,Like a Rolling Stone,Bob Dylan,'classic rock',72,68
1,1,0.485,0.863,1,-9.027,1,0.0495,0.000012,0.016200,0.1380,...,spotify:track:3oTlkzk1OtrhH8wBAduVEi,https://api.spotify.com/v1/tracks/3oTlkzk1Otrh...,https://api.spotify.com/v1/audio-analysis/3oTl...,300977,4,Smells Like Teen Spirit,Nirvana,'grunge',82,0
2,2,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,...,spotify:track:3ZFBeIyP41HhnALjxWy1pR,https://api.spotify.com/v1/tracks/3ZFBeIyP41Hh...,https://api.spotify.com/v1/audio-analysis/3ZFB...,337413,4,A Day In The Life - Remastered,The Beatles,'beatlesque',84,0
3,3,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,...,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...,219147,4,Good Vibrations (Mono),The Beach Boys,'baroque pop',78,20
4,4,0.518,0.756,10,-10.851,1,0.0915,0.735000,0.000062,0.3170,...,spotify:track:7MH2ZclofPlTrZOkPzZKhK,https://api.spotify.com/v1/tracks/7MH2ZclofPlT...,https://api.spotify.com/v1/audio-analysis/7MH2...,160893,4,Johnny B Goode,Chuck Berry,'classic rock',75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,1094,0.509,0.680,5,-8.354,1,0.0372,0.360000,0.000000,0.0824,...,spotify:track:7tJQ4Ekp2vN3NlI3vJJW3v,https://api.spotify.com/v1/tracks/7tJQ4Ekp2vN3...,https://api.spotify.com/v1/audio-analysis/7tJQ...,185200,4,I Want You,Bob Dylan,'classic rock',72,56
1095,1095,0.612,0.668,6,-6.143,0,0.0302,0.057900,0.044500,0.3410,...,spotify:track:6EaHDsTWNNJRkCMtptSE6n,https://api.spotify.com/v1/tracks/6EaHDsTWNNJR...,https://api.spotify.com/v1/audio-analysis/6EaH...,315200,4,The Suburbs,Arcade Fire,'baroque pop',66,0
1096,1096,0.651,0.944,2,-5.480,1,0.0381,0.085500,0.145000,0.1160,...,spotify:track:5mWhSQfWEeBZD9YTd4yyFc,https://api.spotify.com/v1/tracks/5mWhSQfWEeBZ...,https://api.spotify.com/v1/audio-analysis/5mWh...,263440,4,Dum Surfer,King Krule,'uk alternative pop',60,3
1097,1097,0.492,0.194,6,-14.073,1,0.0577,0.984000,0.104000,0.1820,...,spotify:track:5iltwljBW1H5ScLeeiB9ZB,https://api.spotify.com/v1/tracks/5iltwljBW1H5...,https://api.spotify.com/v1/audio-analysis/5ilt...,203373,4,Black and Tan Fantasy,Duke Ellington & His Washingtonians,'hot jazz',18,11


In [189]:
clean_data= sample_p.drop(columns=['Unnamed: 0','uri','track_href','analysis_url','type', 'id'])
clean_data
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1099 non-null   float64
 1   energy            1099 non-null   float64
 2   key               1099 non-null   int64  
 3   loudness          1099 non-null   float64
 4   mode              1099 non-null   int64  
 5   speechiness       1099 non-null   float64
 6   acousticness      1099 non-null   float64
 7   instrumentalness  1099 non-null   float64
 8   liveness          1099 non-null   float64
 9   valence           1099 non-null   float64
 10  tempo             1099 non-null   float64
 11  duration_ms       1099 non-null   int64  
 12  time_signature    1099 non-null   int64  
 13  song_name         1099 non-null   object 
 14  artist_name       1099 non-null   object 
 15  artist_genres     1099 non-null   object 
 16  artist_pop        1099 non-null   int64  


In [190]:
#Create genre list
def genre_list(self):
    split = self.strip().split(',')
    clean = [x.strip(" [']") for x in split]
    return clean

In [191]:
# convert the df to list
clean_data['artist_genres'] = clean_data['artist_genres'].apply(lambda x: genre_list(x))

In [192]:
#finding all the genres in the data set
all_genres = []
for row in clean_data.artist_genres:
    for x in row:
        if x not in all_genres:
            all_genres.append(x)
        
print(all_genres)
    

['classic rock', 'grunge', 'beatlesque', 'baroque pop', 'classic soul', 'adult standards', 'british invasion', 'pop', 'rock-and-roll', 'album rock', 'alternative rock', 'atl hip hop', 'punk', 'dance pop', 'bronx hip hop', 'art rock', 'funk', 'acid rock', 'classic garage rock', 'madchester', 'downtempo', 'conscious hip hop', 'art punk', 'east coast hip hop', 'art pop', 'alternative dance', 'bubblegum pop', 'classic girl group', 'lilith', 'glam metal', 'blues rock', 'chicago rap', 'electro', 'piano rock', 'irish rock', 'europop', 'disco', 'hip hop', 'british soul', 'reggae', 'french shoegaze', 'honky tonk', 'barbadian pop', 'mellow gold', 'candy pop', 'canadian hip hop', 'soul', '', 'american folk revival', 'garage rock', 'australian psych', 'afrofuturism', 'g funk', 'new romantic', 'australian dance', 'brooklyn indie', 'dusseldorf electronic', 'britpop', 'arkansas country', 'mod revival', 'blues', 'chicago soul', 'boston rock', 'british blues', 'alternative metal', 'bossa nova', 'brill 

In [193]:
clean_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop,song_popularity
0,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,0.557,95.263,369600,4,Like a Rolling Stone,Bob Dylan,[classic rock],72,68
1,0.485,0.863,1,-9.027,1,0.0495,0.000012,0.016200,0.1380,0.767,116.835,300977,4,Smells Like Teen Spirit,Nirvana,[grunge],82,0
2,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,0.175,163.219,337413,4,A Day In The Life - Remastered,The Beatles,[beatlesque],84,0
3,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,0.331,133.574,219147,4,Good Vibrations (Mono),The Beach Boys,[baroque pop],78,20
4,0.518,0.756,10,-10.851,1,0.0915,0.735000,0.000062,0.3170,0.968,166.429,160893,4,Johnny B Goode,Chuck Berry,[classic rock],75,0


In [37]:
#combining genres with music feature data average into a new df
genre_features_input = {}
genre_features = {}
#list of features were looking for
features_input = ['danceability','energy','loudness','speechiness','acousticness','liveness','valence']


#loop through df
for i, r in clean_data.iterrows():

    #loop through each genre
    for genre in r['artist_genres']:

        #create a dict for the genre if it isnt made
        if genre not in genre_features_input.keys():
            genre_features_input[genre] = []

            #start adding the features
            for feature in features_input:

                #create dict for features
                if feature not in genre_features.keys():
                    genre_features[feature] = []
                    genre_features[feature].append(r[feature])
                
            genre_features_input[genre].append(genre_features)
        #else add onto the generated genre key
        else:
            for genre_info in genre_features_input.values():
                for dicts in genre_info:
                    for key,v in dicts.items():
                        for feature in features_input:
                            if key == feature:
                                dicts[key].append(r[feature])
                    
                    
                
        genre_features = {}

In [233]:
genre_df =clean_data[['artist_genres', 'danceability','energy','acousticness','valence','song_popularity','artist_pop']].copy()
genre_df

,artist_genres,danceability,energy,acousticness,valence,song_popularity,artist_pop
0,[classic rock],0.482,0.721,0.731000,0.557,68,72
1,[grunge],0.485,0.863,0.000012,0.767,0,82
2,[beatlesque],0.364,0.457,0.290000,0.175,0,84
3,[baroque pop],0.398,0.413,0.082200,0.331,20,78
4,[classic rock],0.518,0.756,0.735000,0.968,0,75
...,...,...,...,...,...,...,...
1094,[classic rock],0.509,0.680,0.360000,0.802,56,72
1095,[baroque pop],0.612,0.668,0.057900,0.524,0,66
1096,[uk alternative pop],0.651,0.944,0.085500,0.676,3,60
1097,[hot jazz],0.492,0.194,0.984000,0.368,11,18


In [234]:
genre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   artist_genres    1099 non-null   object 
 1   danceability     1099 non-null   float64
 2   energy           1099 non-null   float64
 3   acousticness     1099 non-null   float64
 4   valence          1099 non-null   float64
 5   song_popularity  1099 non-null   int64  
 6   artist_pop       1099 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 60.2+ KB


In [236]:
genre_df['artist_genres']  = (genre_df['artist_genres'].astype('string'))
newframe=genre_df.groupby('artist_genres')['danceability','energy','acousticness','valence','song_popularity','artist_pop'].mean()
newframe.info


<bound method DataFrame.info of                         danceability    energy  acousticness   valence  \
artist_genres                                                            
['']                        0.572286  0.384957      0.644897  0.527571   
['acid house']              0.714000  0.873000      0.152000  0.516000   
['acid rock']               0.427800  0.687600      0.237189  0.478880   
['acoustic blues']          0.537667  0.242100      0.958500  0.496333   
['acoustic punk']           0.701000  0.261000      0.219000  0.890000   
...                              ...       ...           ...       ...   
['ska']                     0.697000  0.592500      0.092950  0.902500   
['soul']                    0.516000  0.150000      0.130000  0.205000   
['space age pop']           0.699000  0.503000      0.775000  0.846000   
['surf music']              0.461000  0.955000      0.111000  0.725000   
['uk alternative pop']      0.651000  0.944000      0.085500  0.676000   

     

In [244]:
df = pd.DataFrame(newframe)
new_genre= df.nlargest(10, 'song_popularity',keep='last')
#new_genre.drop_duplicates('artist_genres')
new_genre.info()
new_genre.reset_index(inplace=True)
new_genre2= df.nlargest(10, 'artist_pop',keep='last')
new_genre.info()
new_genre2.reset_index(inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, ['permanent wave'] to ['celtic punk']
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   danceability     10 non-null     float64
 1   energy           10 non-null     float64
 2   acousticness     10 non-null     float64
 3   valence          10 non-null     float64
 4   song_popularity  10 non-null     float64
 5   artist_pop       10 non-null     float64
dtypes: float64(6)
memory usage: 560.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   artist_genres    10 non-null     string 
 1   danceability     10 non-null     float64
 2   energy           10 non-null     float64
 3   acousticness     10 non-null     float64
 4   valence          10 non-null     float64
 5   song_popularity  10 non-null    

In [238]:
new_genre

,artist_genres,danceability,energy,acousticness,valence,song_popularity,artist_pop
0,['permanent wave'],0.478000,0.690286,0.230535,0.481143,81.428571,84.714286
1,['rap'],0.834000,0.730000,0.005130,0.446000,81.000000,91.000000
2,['modern rock'],0.542000,0.905000,0.001720,0.374000,80.000000,74.000000
3,['lgbtq+ hip hop'],0.878000,0.619000,0.053300,0.639000,77.000000,83.000000
4,['folk'],0.711000,0.292000,0.313000,0.194000,76.000000,69.000000
5,['glam punk'],0.535000,0.716000,0.326000,0.901000,75.000000,64.000000
6,['madchester'],0.510667,0.848333,0.015315,0.784000,74.666667,76.000000
7,['bedroom pop'],0.742000,0.546000,0.172000,0.868000,73.000000,76.000000
8,['modern alternative rock'],0.472000,0.745000,0.000607,0.063100,71.000000,82.000000
9,['celtic punk'],0.271000,0.551000,0.582000,0.428000,71.000000,72.000000


In [245]:
new_genre2

,artist_genres,danceability,energy,acousticness,valence,song_popularity,artist_pop
0,['canadian hip hop'],0.836000,0.518000,0.003360,0.436000,0.000000,97.000000
1,['canadian contemporary r&b'],0.702000,0.770000,0.124000,0.593000,0.000000,96.000000
2,['detroit hip hop'],0.820000,0.715000,0.031750,0.537525,0.000000,92.000000
3,['rap'],0.834000,0.730000,0.005130,0.446000,81.000000,91.000000
4,['chicago rap'],0.597857,0.701000,0.210621,0.399971,20.142857,91.000000
5,['barbadian pop'],0.680667,0.709667,0.036280,0.577667,0.000000,90.000000
6,['garage rock'],0.541500,0.740000,0.094125,0.591500,0.000000,88.000000
7,['glam rock'],0.533000,0.414000,0.554750,0.317250,0.000000,87.000000
8,['r&b en espanol'],0.893000,0.482000,0.549000,0.530000,67.000000,86.000000
9,['pop'],0.818333,0.735333,0.092400,0.823667,61.166667,85.666667


In [252]:
#top10_genres= genre_df.nlargest(10, 'song_popularity')

fig = px.bar(new_genre, x='artist_genres', y=['valence', 'energy', 'danceability', 'acousticness'], barmode='group')


fig.update_layout(
    title="Genre Poularity Based on Song",
    yaxis = dict(
        tickmode = 'linear',
        title="Genre Poularity Based on Song",
        tick0 = 0,
        dtick = 0.2
    )
)
fig.show()
        

In [250]:
fig = px.bar(new_genre2, x='artist_genres', y=['valence', 'energy', 'danceability', 'acousticness'], barmode='group')


fig.update_layout(
    title="Genre Poularity Based on Artist",
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.2
    )
)
fig.show()

In [206]:
cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=10,random_state =0))])
X = genre_df.select_dtypes(np.number)
cluster_pipeline.fit(X)
genre_df['cluster'] = cluster_pipeline.predict(X)

In [209]:
tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=1))])
genre_embedding = tsne_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = genre_df['artist_genres']
projection['cluster'] = genre_df['cluster']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1099 samples in 0.002s...
[t-SNE] Computed neighbors for 1099 samples in 0.041s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1099
[t-SNE] Computed conditional probabilities for sample 1099 / 1099
[t-SNE] Mean sigma: 0.622092
[t-SNE] KL divergence after 250 iterations with early exaggeration: 56.649952
[t-SNE] KL divergence after 1000 iterations: 0.598758


In [230]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=10, 
                                   verbose=False, random_state=42))
                                 ], verbose=False)

X = clean_data.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
clean_data['cluster_label'] = song_cluster_labels

In [231]:
pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = clean_data['song_name']
projection['cluster'] = clean_data['cluster_label']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

In [253]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'd94b094580d74927972cca8096281e2d'
secret = '727603b3837c44b98b1cbb1e876c4c08'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [263]:
def find_song(name, year):
    song_data = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,year), limit=1)
    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)


In [265]:
from collections import defaultdict
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']


def get_song_data(song, spotify_data):
    
    try:
        song_data = spotify_data[(spotify_data['name'] == song['name']) 
                                & (spotify_data['year'] == song['year'])].iloc[0]
        return song_data
    
    except IndexError:
        return find_song(song['name'], song['year'])
        

def get_mean_vector(song_list, spotify_data):
    
    song_vectors = []
    
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)


def flatten_dict_list(dict_list):
    
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict

In [266]:
def recommend_songs( song_list, spotify_data, n_songs=10):
    
    metadata_cols = ['name', 'year', 'artists']
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, spotify_data)
    scaler = song_cluster_pipeline.steps[0][1]
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols].to_dict(orient='records')

In [269]:
 song_dict = flatten_dict_list(song_list)

NameError: name 'song_list' is not defined

In [267]:
recommend_songs([{'song_name': 'Come As You Are', 'year':1991},
                {'song_name': 'Smells Like Teen Spirit', 'year': 1991},
                {'song_name': 'Lithium', 'year': 1992},
                {'song_name': 'All Apologies', 'year': 1993},
                {'song_name': 'Stay Away', 'year': 1993}], sample_p)

KeyError: 'name'